# A Cloud Price Comparison
How do cloud providers stack up?
 
Making apples to apples comparisons between different cloud providers is very difficult, because each one offers instances with varying vCPUs, RAM, SSD space and HDD space. To further obfuscate matters, slightly different billing systems, promises of arcane discounting, only providing pricing in USD, and inconsistent naming conventions are sprinkled throughout.

As an attempt to provide a clearer price comparison, I'll be using [multiple linear regression](https://en.wikipedia.org/wiki/Linear_regression) to "[normalise](http://bit.ly/2xIUM5C)" the pricing of compute instances across different cloud providers.

In essence, **If every cloud provider offered the same size compute instances, how expensive would they be?**

## Todo

* Account for GST

 ## Importing libraries

In [1]:
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import requests
import json
import re
from bs4 import BeautifulSoup
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## The dataset

I'll be taking the price tables of:
* Google Cloud - [Predefined machine types](https://cloud.google.com/compute/pricing#predefined_machine_types)
* AWS - [On demand instances](https://aws.amazon.com/ec2/pricing/on-demand/)
* Azure - [Linux virtual machines](https://azure.microsoft.com/en-us/pricing/details/virtual-machines/linux/)

and converting them into the instance sizes offered by [Catalyst Cloud](https://www.catalyst.net.nz/catalyst-cloud/prices). You can find the datasets and their sources [here](https://github.com/catalyst-cloud/catalystcloud-price-comparison/raw/master/dataset/Cloud%20price%20comparison.ods).

In [2]:
# The list of URLS

# Catalyst Cloud's price list
catalyst_url = 'https://catalystcloud.nz/services/iaas/compute'
catalyst_data_location = 'dataset/catalyst_price_data.csv'

# AWS's bulk price list JSON, for the Sydney region (ap-southeast-2)
aws_url = 'https://aws.amazon.com/ec2/pricing/on-demand/'
aws_raw_location = 'dataset/raw_aws_bulk.json'
aws_acceptable_instance_families = [
    'General purpose',
    'Memory optimized',
    'Micro instances'
]
aws_data_location = 'dataset/aws_price_data.csv'

# Google Cloud's prices
google_url = 'https://cloud.google.com/compute/pricing'
google_price_type = 'syd-hourly'
google_acceptable_instance_families = [
    'standard_machine_types',
    'high-memory_machine_types',
    'high-cpu_machine_types',
    
]
google_data_location = 'dataset/google_price_data.csv'

# Azure
azure_url = 'http://www.azureinstances.info/api/Compute/Get?delimitedRegions=Australia Southeast'
azure_raw_location = 'dataset/raw_azure_bulk.json'
azure_data_location = 'dataset/azure_price_data.csv'

# List of existing dataset files
datasets = os.listdir('dataset')

fresh = False

### Exchange rate
Getting the USD to NZD exchange rate

In [3]:
usd_to_nzd_exchange_rate_url = 'http://free.currencyconverterapi.com/api/v5/convert?q=USD_NZD&compact=y'

usd_to_nzd_exchange_rate_json = requests.get(usd_to_nzd_exchange_rate_url).json()
usd_to_nzd_exchange_rate = float(usd_to_nzd_exchange_rate_json['USD_NZD']['val'])

### Catalyst Cloud prices
Scraping the Catalyst Cloud compute page for prices

In [4]:
catalyst_price_page_html = requests.get(catalyst_url).text 
catalyst_price_page = BeautifulSoup(catalyst_price_page_html, 'html.parser')

catalyst_price_table = catalyst_price_page.find(attrs={'class': 'service-price-table'}).tbody
catalyst_price_rows = catalyst_price_table.find_all('tr')

catalyst_prices_list = []

for row in catalyst_price_rows:
    catalyst_price_cells = list(row.stripped_strings)
    catalyst_prices_list.append({
        'Name': catalyst_price_cells[0],
        'vCPU': float(catalyst_price_cells[1]),
        'RAM, GB': float(catalyst_price_cells[2]),
        'Price per hour, NZD (ex GST)': float(catalyst_price_cells[3].strip('$')),
        'SSD storage, GB': .0,
        'HDD storage, GB': .0
    })
    
pd.DataFrame(catalyst_prices_list).to_csv(catalyst_data_location)

### AWS prices

Scraping the AWS on demand pricing page for prices

In [5]:
if aws_raw_location.split('/')[1] not in datasets or fresh:
    aws_bulk_json_requeset = requests.get(aws_url)
    aws_bulk_json = aws_bulk_json_requeset.json()
    with open(aws_raw_location, 'w') as aws_raw_file:
        json.dump(aws_bulk_json, aws_raw_file)
    print('Downloaded most recent AWS price list.')
else:
    print('AWS raw dataset already dowloaded.')

AWS raw dataset already dowloaded.


In [6]:
with open(aws_raw_location, 'r') as aws_raw_file:
    aws_raw_json = json.load(aws_raw_file)


Extracting the specific relevant prices from the raw AWS file, and putting them in a consistant, usable format.

In [7]:
# Getting the instance products

with open(aws_raw_location, 'r') as aws_raw_file:
    aws_raw_json = json.load(aws_raw_file)
        
    aws_instances_list = []
            
    for product in aws_raw_json['products']:
        
        productFamily = aws_raw_json['products'][product]['productFamily']
        
        # Check product is compute instance
        if productFamily == 'Compute Instance':
                        
            # Check if instance is appropriate
            instanceFamily = aws_raw_json['products'][product]['attributes']['instanceFamily']
            is_current_gen = aws_raw_json['products'][product]['attributes']['currentGeneration'] == 'Yes'
            is_linux = aws_raw_json['products'][product]['attributes']['operatingSystem'] == 'Linux'
            no_preInstalledSw = aws_raw_json['products'][product]['attributes']['preInstalledSw'] == 'NA'
            is_shared_instance = aws_raw_json['products'][product]['attributes']['tenancy'] == 'Shared'

            if instanceFamily in aws_acceptable_instance_families and is_current_gen \
                and is_linux and no_preInstalledSw and is_shared_instance:
                
                # Append if appropriate
                aws_instances_list.append(product)

In [8]:
with open(aws_raw_location, 'r') as aws_raw_file:
    
    aws_prices_list = []
    
    for instance_key in aws_instances_list:

        attributes = aws_raw_json['products'][instance_key]['attributes']
                    
        # Get vCPU and RAM
        vCPU = float(attributes['vcpu'].replace(',',''))
        RAM = float(attributes['memory'].split(' ')[0].replace(',',''))

        # Break storage spec into array
        storage_strings = attributes['storage'].split(' ')

        # Find where the numbers end (200 x 1), and the description of the storage type (SSD) starts.
        final_num_index = None
        for word in storage_strings[::-1]:
            try:
                float(word.replace(',', ''))
                final_num_index = storage_strings.index(word)
                break
            except:
                foo = None

        # If there are no numbers in the storage spec, there is no storage included
        if final_num_index == None:

            total_ssd = .0
            total_hdd = .0

        # Else...
        else:

            # Perform the math to figure out how many GB of storage is included
            storage_calcs = storage_strings[0:final_num_index+1]
            storage_volume = eval(' '.join(['*' if x=='x' else x.replace(',', '') for x in storage_calcs]))

            # discern the type of storage
            if 'HDD' in storage_strings:                        
                total_ssd = .0
                total_hdd = float(storage_volume)

            elif 'SSD' in storage_strings:                        
                total_ssd = float(storage_volume)
                total_hdd = .0
            else: 
                total_ssd = float(storage_volume)
                total_hdd = .0


        # Get the price per USD
        terms = aws_raw_json['terms']['OnDemand'][instance_key]
        usd_price = None
        for specific_term in terms:
            for dimension_key in terms[specific_term]['priceDimensions']:
                dimension = terms[specific_term]['priceDimensions'][dimension_key]
                if dimension['unit'] != 'Hrs': raise ValueError("This price isn't in hours")
                usd_price = float(dimension['pricePerUnit']['USD'])

        # Convert to NZD
        nzd_price = usd_price * usd_to_nzd_exchange_rate
                
        # Append to list of prices
        aws_prices_list.append({
            'Name': attributes['instanceType'],
            'vCPU': vCPU,
            'RAM, GB': RAM,
            'Price per hour, NZD (ex GST)': nzd_price,
            'SSD storage, GB': total_ssd,
            'HDD storage, GB': total_hdd
        })

# Convert to CSV
pd.DataFrame(aws_prices_list).to_csv(aws_data_location)

### Google Cloud prices

Scraping Google Cloud's documentation for prices of custom instance sizes.

In [9]:
google_price_page_html = requests.get(google_url).text
google_price_page = BeautifulSoup(google_price_page_html, 'html.parser')

In [10]:
# Extract the USD price per vCPU and per GB RAM
google_custom_compute_price_table = google_price_page.find(id='custommachinetypepricing').find_next('table')
google_custom_compute_rows = google_custom_compute_price_table.find_all('tr')[1:]

google_per_vcpu_usd = float(google_custom_compute_rows[0].find_all('td')[1][google_price_type].split()[0].strip('$'))
google_per_ram_usd = float(google_custom_compute_rows[1].find_all('td')[1][google_price_type].split()[0].strip('$'))

In [11]:
def most_freq_num(text):
    number_list = re.findall('\d*\.?\d+', text)
    most_frequent_num = max(set(number_list), key=number_list.count)
    return float(most_frequent_num)

In [12]:

google_prices_list = []

for instance_type in google_acceptable_instance_families:
    
    google_price_table = google_price_page.find(id=instance_type).find_next('table')
    google_rows = google_price_table.find_all('tr')[1:-1]
        
    for row in google_rows:
        
        # Extract number of vCPUs and GB of RAM
        try:
            cells = row.find_all('td')
            name = cells[0].get_text().strip()
            # Ignore if has lake in name (to remove skylake instances)
            if 'lake' in name:
                continue
            cpu_val = most_freq_num(str(cells[1]))
            ram_val = most_freq_num(str(cells[2]))
        except:
            foo='bar'
                        
        # Calcluate NZD price
        usd_price = (google_per_ram_usd * ram_val) + (google_per_vcpu_usd * cpu_val)
        nzd_price = usd_price * usd_to_nzd_exchange_rate
        
        try:
            google_prices_list.append({
                'Name': name,
                'vCPU': cpu_val,
                'RAM, GB': ram_val,
                'Price per hour, NZD (ex GST)': usd_price,
                'SSD storage, GB': .0,
                'HDD storage, GB': .0
            })
        except:
            continue

pd.DataFrame(google_prices_list).to_csv(google_data_location)

### Azure cloud prices
Get Azure pricing from community maintained price list. This is because it is too difficult to scrape the Azure pricing page, and they maintain no APIs.

In [13]:
# azure_price_xml =  requests.get(azure_url).content

if azure_raw_location.split('/')[1] not in datasets or fresh:
    azure_bulk_json_requeset = requests.get(azure_url)
    azure_bulk_json = azure_bulk_json_requeset.json()
    with open(azure_raw_location, 'w') as azure_raw_file:
        json.dump(azure_bulk_json, azure_raw_file)
    print('Downloaded most recent Azure price list.')
else:
    print('Azure raw dataset already dowloaded.')

Azure raw dataset already dowloaded.


### Ingesting the datasets
Importing the datasets.

In [ ]:
catalyst_dataset = pd.read_csv("dataset/Rigorious Cloud price comparison - Exportable Catalyst prices.csv")
google_dataset = pd.read_csv("dataset/Rigorious Cloud price comparison - Exportable Google prices.csv")
aws_dataset = pd.read_csv("dataset/Rigorious Cloud price comparison - Exportable AWS prices.csv")
azure_dataset = pd.read_csv("dataset/Rigorious Cloud price comparison - Exportable Azure prices.csv")

Previewing the datasets.

In [ ]:
catalyst_dataset.head(6)

Now we'll split the data into NumPy arrays of input features (X) and labels (Y).

In [ ]:
def split_dataset (dataset):
    x = dataset[["vCPUs", "RAM (GiB)", "HDD (GB)", "SSD (GB)"]].values
    y = dataset["Price per hour (NZD)"].values
    
    return (x, y)

In [ ]:
catalyst_x, catalyst_y = split_dataset(catalyst_dataset)
google_x, google_y = split_dataset(google_dataset)
aws_x, aws_y = split_dataset(aws_dataset)
azure_x, azure_y = split_dataset(azure_dataset)

## The math
To analyse this dataset, we'll be using [multiple linear regression](https://en.wikipedia.org/wiki/Linear_regression) to predict the prices of compute instance flavors if they were being offered by cloud providers that do not typically offer that sized flavor.

The multiple linear regression models will draw a hyperplane across our data space (in this case, 5 dimensional space) that comes as close as possible to intersecting every data point in our dataset. You can see an example of this (in 2 dimensional space) by Khan academy below.

In [ ]:
from IPython.core.display import display, HTML
display(HTML('<iframe width="840" height="472" src="https://www.youtube-nocookie.com/embed/GAmzwIkGFgE?rel=0" frameborder="0" allowfullscreen></iframe>'))

By then finding a point on the hyperplane that intersects with our vCPUs, RAM, HDD, and SSD axes, we can find the predicted price. This will give us a way to estimate the price of a flavour if it were offered by various cloud providers, even if they do not offer it.

I've used linear regression as the predictive algorithm because I'm assuming that cloud providers scale their pricing in a linear patern. For example, Catalyst Cloud decides the price of their flavors with the equation: 

Price/hour = (RAM gb x 0.18) + (vCPU x 0.026)

First we'll initialise the regression models, and train them on the cloud providers' prices.

In [ ]:
# Initialise regressors
catalyst_linear = LinearRegression()
google_linear = LinearRegression()
aws_linear = LinearRegression()
azure_linear = LinearRegression()

# Train regressors
catalyst_linear.fit(catalyst_x, catalyst_y)
google_linear.fit(google_x, google_y)
aws_linear.fit(aws_x, aws_y)
azure_linear.fit(azure_x, azure_y)

Now we have the models predict the other providers' instance prices. By having all providers predict the prices of all other providers, we can see if the pattern is maintained across different models.

In [ ]:
# Predict Catalyst X
google_cata_price = google_linear.predict(catalyst_x)
aws_cata_price = aws_linear.predict(catalyst_x)
azure_cata_price = azure_linear.predict(catalyst_x)

# Predict Google X
aws_goog_price = aws_linear.predict(google_x)
azure_goog_price = azure_linear.predict(google_x)
catalyst_goog_price = catalyst_linear.predict(google_x)

# Predict AWS X
google_aws_price = google_linear.predict(aws_x)
azure_aws_price = azure_linear.predict(aws_x)
catalyst_aws_price = catalyst_linear.predict(aws_x)

# Predict Azure X
google_azure_price = google_linear.predict(azure_x)
aws_azure_price = aws_linear.predict(azure_x)
catalyst_azure_price = catalyst_linear.predict(azure_x)

## The results

Now we have the results together, where we can compare the prices against each other on an even scale.

A good scientist would, at this point, verify their results by comparing an intersection between the predicted output and the actual output. I would love to do this. However I could find no such intersection.

You can find the datasets this analysis is based on [here](https://github.com/catalyst-cloud/catalystcloud-price-comparison/raw/master/dataset/Cloud%20price%20comparison.ods), and a chart plotting this data, [here](https://object-storage.nz-por-1.catalystcloud.io/v1/AUTH_8ccc3286887e49cb9a40f023eba693b4/catalyst-cloud-price-comp/).

Please note that the X axis is a range from zero to the number of flavors offered by each provider. Each number on the X axis represents a single flavor by the provider we are predicting. I've done it this way because the plotting method does not support non-numerical axis ticks.

In [ ]:
def graph_pred (names, predictions):
    flavors_num = predictions[0].shape[0]
    for index, name in enumerate(names):
        plt.plot(range(flavors_num), predictions[index], label=names[index])
    plt.legend(loc=2)

### Predicting Catalyst Cloud

In [ ]:
graph_pred([
        "Catalyst", "Google", "AWS", "Azure"
    ], [
        catalyst_y, google_cata_price, aws_cata_price, azure_cata_price
    ])

### Predicting Google Cloud

In [ ]:
graph_pred([
        "Catalyst", "Google", "AWS", "Azure"
    ], [
        catalyst_goog_price,
        google_y,
        aws_goog_price,
        azure_goog_price,
        
    ])

### Predicting AWS

In [ ]:
graph_pred([
        "Catalyst", "Google", "AWS", "Azure"
    ], [
        catalyst_aws_price,
        google_aws_price,
        aws_y,
        azure_aws_price,
    ])

### Predicting Azure

In [ ]:
graph_pred([
        "Catalyst", "Google", "AWS", "Azure"
    ], [
        catalyst_azure_price,
        google_azure_price,
        aws_azure_price,
        azure_y
    ])

### Interpreting the results

As you can see, the multiple linear regression model consistently predicts that Catalyst Cloud would offer either the cheapest, or competitively priced compute instances.

While this is by no means a perfect indicator of who is the cheapest and should not be taken as such, it does serve to dispel the idea that Catalyst Cloud is overpriced, or cannot compete with international companies on price.